In [1]:
import pandas as pd
import numpy as np

In [3]:
fpkm = pd.read_csv('E:\\deeplearning\\Hepatocarcinomes\\TCGA\\processed\\fpkm_TCGA.csv', sep=',')
firehose_data = pd.read_csv('E:\\deeplearning\\Hepatocarcinomes\\TCGA\\lihc_tcga_firehose\\firehose_mRNA_RSEM_normalized_data_clean.csv', sep="\t", encoding='utf8', engine='python')
ref0 = pd.read_csv('E:\\deeplearning\\Hepatocarcinomes\\TCGA\\additional_ref_Emsembl.txt', sep="\t", encoding='utf8', engine='python')
filter_data = pd.read_csv('E:\\deeplearning\\Hepatocarcinomes\\TCGA\\ProbeAnnotations_NS_IO_360_v1.0_clean.csv', sep='\t')

In [31]:
fpkm.rename(columns={'Unnamed: 0':'Gene Name'}, inplace=True)

In [39]:
firehose747 = pd.read_csv('E:\\deeplearning\\Hepatocarcinomes\\TCGA\\processed\\firebrowse_filtered_nohousekeeping_HCC.csv', sep='\t')

In [42]:
firehose747["Gene Name"]

0        CDKN2A
1          GBP2
2          TLR8
3          SNCA
4         BIRC3
         ...   
742    C6ORF150
743      MRE11A
744        EMR1
745      DEPDC6
746         IL8
Name: Gene Name, Length: 747, dtype: object

In [37]:
len(set(i for i in fpkm["Gene Name"].values.tolist()))

753

In [53]:
set(i for i in fpkm["Gene Name"].values.tolist()) - set(i for i in firehose747["Gene Name"].values.tolist()) - set(filter_data[filter_data["Control.Type"]=="Housekeeping"]["Probe.Label"].values)

{'CCL3', 'CXorf36'}

In [29]:
fpkm['AACA-02A']

0      386.113618
1       15.834714
2        0.210662
3        0.043093
4        4.354185
          ...    
748      0.000000
749      0.058683
750      2.854628
751      2.004253
752      0.247051
Name: AACA-02A, Length: 753, dtype: float64

In [3]:
print(firehose_data.shape)
display(firehose_data.head(8))

(20531, 426)


,Gene Name,Gene ID,Hybridization REF,TCGA-2V-A95S-01A-11R-A37K-07 normalized_count,TCGA-2Y-A9GS-01A-12R-A38B-07 normalized_count,TCGA-2Y-A9GT-01A-11R-A38B-07 normalized_count,TCGA-2Y-A9GU-01A-11R-A38B-07 normalized_count,TCGA-2Y-A9GV-01A-11R-A38B-07 normalized_count,TCGA-2Y-A9GW-01A-11R-A38B-07 normalized_count,TCGA-2Y-A9GX-01A-11R-A38B-07 normalized_count,...,TCGA-ZP-A9CZ-01A-11R-A38B-07 normalized_count,TCGA-ZP-A9D0-01A-11R-A37K-07 normalized_count,TCGA-ZP-A9D1-01A-11R-A38B-07 normalized_count,TCGA-ZP-A9D2-01A-11R-A38B-07 normalized_count,TCGA-ZP-A9D4-01A-11R-A37K-07 normalized_count,TCGA-ZS-A9CD-01A-11R-A37K-07 normalized_count,TCGA-ZS-A9CE-01A-11R-A37K-07 normalized_count,TCGA-ZS-A9CF-01A-11R-A38B-07 normalized_count,TCGA-ZS-A9CF-02A-11R-A38B-07 normalized_count,TCGA-ZS-A9CG-01A-11R-A37K-07 normalized_count
0,?,100130426,?|100130426,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,?,100133144,?|100133144,1.5051,26.4120,0.0000,5.7222,11.4975,3.9234,0.0000,...,3.8367,3.9575,2.2186,15.9098,1.0069,4.2224,0.0000,7.3398,8.3127,0.0000
2,?,100134869,?|100134869,3.7074,2.6663,4.4833,5.1216,5.4230,7.5709,8.4848,...,4.8549,2.4775,4.8553,18.6608,6.9127,1.7881,8.7527,9.8897,11.4648,8.2262
3,?,10357,?|10357,90.1124,71.0054,95.5122,61.6679,104.4670,99.4866,75.0909,...,93.5837,62.9730,73.0161,65.1237,71.2827,64.5605,64.9891,105.8856,108.7824,57.1654
4,?,10431,?|10431,1017.1038,639.2311,742.4344,1186.9807,878.1726,690.4215,680.6061,...,905.7892,1259.9743,942.7653,1008.7336,685.0516,1087.9038,1119.2560,544.4521,676.1434,917.9092
5,?,136542,?|136542,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
6,?,155060,?|155060,141.3911,122.7206,95.0460,280.2709,282.5719,68.9655,124.8485,...,135.9615,186.6152,111.8971,420.6696,24.3247,41.3223,35.0109,262.5775,314.5859,96.6581
7,?,26823,?|26823,0.6516,1.4786,1.7933,0.8341,0.0000,0.0000,0.0000,...,1.2417,0.6435,2.5723,0.3639,0.0000,0.0000,0.0000,0.0000,1.2361,0.0000


In [4]:
firehose_data["Hybridization REF"].isna().sum()

0

In [5]:
# Delete the first column -- ENSG ID (could be multiple for 1 gene)
ref0.drop(columns=["Gene stable ID version"], inplace=True)
ref0.drop_duplicates(inplace=True)
print(ref0.shape)
display(ref0.head(8))
print(ref0.isna().sum())

(767, 3)


,Gene name,NCBI gene ID,RefSeq mRNA ID
0,A2M,2,NM_000014
1,APOE,348,NM_000041
2,FAS,355,NM_000043
3,ARG1,383,NM_000045
4,ATM,472,NM_000051
5,BLM,641,NM_000057
6,BRCA2,675,NM_000059
7,C2,717,NM_000063


Gene name         0
NCBI gene ID      0
RefSeq mRNA ID    0
dtype: int64


In [6]:
buffer = pd.concat([pd.DataFrame(ref0["Gene name"].astype(str)), pd.DataFrame(ref0["NCBI gene ID"].astype(str))], axis=1)
ref0["(Gene name, NCBI gene ID)"] = buffer[["Gene name", "NCBI gene ID"]].agg('|'.join, axis=1)
ref0.head(8)

,Gene name,NCBI gene ID,RefSeq mRNA ID,"(Gene name, NCBI gene ID)"
0,A2M,2,NM_000014,A2M|2
1,APOE,348,NM_000041,APOE|348
2,FAS,355,NM_000043,FAS|355
3,ARG1,383,NM_000045,ARG1|383
4,ATM,472,NM_000051,ATM|472
5,BLM,641,NM_000057,BLM|641
6,BRCA2,675,NM_000059,BRCA2|675
7,C2,717,NM_000063,C2|717


In [7]:
buffer = pd.concat([pd.DataFrame(ref0["Gene name"].astype(str)), pd.DataFrame(ref0["RefSeq mRNA ID"].astype(str))], axis=1)
ref0["(Gene name, RefSeq mRNA ID)"] = buffer[["Gene name", "RefSeq mRNA ID"]].agg('|'.join, axis=1)
ref0.head(8)

,Gene name,NCBI gene ID,RefSeq mRNA ID,"(Gene name, NCBI gene ID)","(Gene name, RefSeq mRNA ID)"
0,A2M,2,NM_000014,A2M|2,A2M|NM_000014
1,APOE,348,NM_000041,APOE|348,APOE|NM_000041
2,FAS,355,NM_000043,FAS|355,FAS|NM_000043
3,ARG1,383,NM_000045,ARG1|383,ARG1|NM_000045
4,ATM,472,NM_000051,ATM|472,ATM|NM_000051
5,BLM,641,NM_000057,BLM|641,BLM|NM_000057
6,BRCA2,675,NM_000059,BRCA2|675,BRCA2|NM_000059
7,C2,717,NM_000063,C2|717,C2|NM_000063


In [8]:
print(filter_data.shape)
display(filter_data.head(8))

(784, 12)


,ProbeID,Codeset.Name,Probe.Label,Analyte.Type,Is.Control,Control.Type,Related.Probes,Probe.Annotation,KEGG.Pathways,Cell.Type,Official.Gene.Name,Control.Conc;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
0,NM_000077.4:1052,NS_IO_360_V1.0,CDKN2A,mRNA,False,NaN,NaN,Cell Proliferation;Metabolic Stress,hsa04110;hsa04115;hsa05166;hsa05200;hsa05203;h...,NaN,CDKN2A,;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;...
1,NM_004120.4:1744,NS_IO_360_V1.0,GBP2,mRNA,False,NaN,NaN,Interferon Signaling,NaN,NaN,GBP2,;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;...
2,NM_138636.4:2210,NS_IO_360_V1.0,TLR8,mRNA,False,NaN,NaN,Myeloid Compartment,hsa04620,NaN,TLR8,;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;...
3,NM_001146055.1:480,NS_IO_360_V1.0,SNCA,mRNA,False,NaN,NaN,NaN,hsa05010;hsa05012,NaN,SNCA,;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;...
4,NM_001165.4:6567,NS_IO_360_V1.0,BIRC3,mRNA,False,NaN,NaN,Apoptosis;NF-kappaB Signaling,hsa04064;hsa04120;hsa04210;hsa04510;hsa04621;h...,NaN,BIRC3,;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;...
5,NM_005516.5:1287,NS_IO_360_V1.0,HLA-E,mRNA,False,NaN,NaN,Antigen Presentation;Immune Cell Adhesion and ...,hsa04144;hsa04145;hsa04514;hsa04612;hsa04650;h...,NaN,HLA-E,;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;...
6,NM_001135652.2:516,NS_IO_360_V1.0,EIF2AK2,mRNA,False,NaN,NaN,Interferon Signaling,hsa04141;hsa05160;hsa05162;hsa05164;hsa05168;h...,NaN,EIF2AK2,;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;...
7,NM_000972.2:657,NS_IO_360_V1.0,RPL7A,mRNA,False,NaN,NaN,Angiogenesis,NaN,NaN,RPL7A,;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;...


In [9]:
mondor = filter_data.filter(items=["ProbeID", "Probe.Label", "Official.Gene.Name"])
print(mondor.shape)
display(mondor.head(8))

(784, 3)


,ProbeID,Probe.Label,Official.Gene.Name
0,NM_000077.4:1052,CDKN2A,CDKN2A
1,NM_004120.4:1744,GBP2,GBP2
2,NM_138636.4:2210,TLR8,TLR8
3,NM_001146055.1:480,SNCA,SNCA
4,NM_001165.4:6567,BIRC3,BIRC3
5,NM_005516.5:1287,HLA-E,HLA-E
6,NM_001135652.2:516,EIF2AK2,EIF2AK2
7,NM_000972.2:657,RPL7A,RPL7A


In [10]:
mondor["ProbeID"] = pd.DataFrame([id.split(".")[0] for id in mondor["ProbeID"]])
print(mondor.shape)
display(mondor.head(8))

(784, 3)


,ProbeID,Probe.Label,Official.Gene.Name
0,NM_000077,CDKN2A,CDKN2A
1,NM_004120,GBP2,GBP2
2,NM_138636,TLR8,TLR8
3,NM_001146055,SNCA,SNCA
4,NM_001165,BIRC3,BIRC3
5,NM_005516,HLA-E,HLA-E
6,NM_001135652,EIF2AK2,EIF2AK2
7,NM_000972,RPL7A,RPL7A


In [11]:
buffer = pd.concat([pd.DataFrame(mondor["Probe.Label"].astype(str)), pd.DataFrame(mondor["ProbeID"].astype(str))], axis=1)
mondor["(Probe.Label, RefSeq mRNA ID)"] = buffer[["Probe.Label", "ProbeID"]].agg('|'.join, axis=1)
mondor.head(8)

,ProbeID,Probe.Label,Official.Gene.Name,"(Probe.Label, RefSeq mRNA ID)"
0,NM_000077,CDKN2A,CDKN2A,CDKN2A|NM_000077
1,NM_004120,GBP2,GBP2,GBP2|NM_004120
2,NM_138636,TLR8,TLR8,TLR8|NM_138636
3,NM_001146055,SNCA,SNCA,SNCA|NM_001146055
4,NM_001165,BIRC3,BIRC3,BIRC3|NM_001165
5,NM_005516,HLA-E,HLA-E,HLA-E|NM_005516
6,NM_001135652,EIF2AK2,EIF2AK2,EIF2AK2|NM_001135652
7,NM_000972,RPL7A,RPL7A,RPL7A|NM_000972


In [12]:
buffer = pd.concat([pd.DataFrame(mondor["Official.Gene.Name"].astype(str)), pd.DataFrame(mondor["ProbeID"].astype(str))], axis=1)
mondor["(Official.Gene.Name, RefSeq mRNA ID)"] = buffer[["Official.Gene.Name", "ProbeID"]].agg('|'.join, axis=1)
mondor.head(8)

,ProbeID,Probe.Label,Official.Gene.Name,"(Probe.Label, RefSeq mRNA ID)","(Official.Gene.Name, RefSeq mRNA ID)"
0,NM_000077,CDKN2A,CDKN2A,CDKN2A|NM_000077,CDKN2A|NM_000077
1,NM_004120,GBP2,GBP2,GBP2|NM_004120,GBP2|NM_004120
2,NM_138636,TLR8,TLR8,TLR8|NM_138636,TLR8|NM_138636
3,NM_001146055,SNCA,SNCA,SNCA|NM_001146055,SNCA|NM_001146055
4,NM_001165,BIRC3,BIRC3,BIRC3|NM_001165,BIRC3|NM_001165
5,NM_005516,HLA-E,HLA-E,HLA-E|NM_005516,HLA-E|NM_005516
6,NM_001135652,EIF2AK2,EIF2AK2,EIF2AK2|NM_001135652,EIF2AK2|NM_001135652
7,NM_000972,RPL7A,RPL7A,RPL7A|NM_000972,RPL7A|NM_000972


In [13]:
mondor1 = mondor.filter(items=["ProbeID", "Probe.Label", "Official.Gene.Name", "(Probe.Label, RefSeq mRNA ID)"])
mondor1.columns = ["ProbeID", "Probe.Label", "Official.Gene.Name", "(Gene name, RefSeq mRNA ID)"]
print(mondor1.shape)
display(mondor1.head(8))

(784, 4)


,ProbeID,Probe.Label,Official.Gene.Name,"(Gene name, RefSeq mRNA ID)"
0,NM_000077,CDKN2A,CDKN2A,CDKN2A|NM_000077
1,NM_004120,GBP2,GBP2,GBP2|NM_004120
2,NM_138636,TLR8,TLR8,TLR8|NM_138636
3,NM_001146055,SNCA,SNCA,SNCA|NM_001146055
4,NM_001165,BIRC3,BIRC3,BIRC3|NM_001165
5,NM_005516,HLA-E,HLA-E,HLA-E|NM_005516
6,NM_001135652,EIF2AK2,EIF2AK2,EIF2AK2|NM_001135652
7,NM_000972,RPL7A,RPL7A,RPL7A|NM_000972


In [14]:
mondor2 = mondor.filter(items=["ProbeID", "Probe.Label", "Official.Gene.Name", "(Official.Gene.Name, RefSeq mRNA ID)"])
mondor2.columns = ["ProbeID", "Probe.Label", "Official.Gene.Name", "(Gene name, RefSeq mRNA ID)"]
print(mondor2.shape)
display(mondor2.head(8))

(784, 4)


,ProbeID,Probe.Label,Official.Gene.Name,"(Gene name, RefSeq mRNA ID)"
0,NM_000077,CDKN2A,CDKN2A,CDKN2A|NM_000077
1,NM_004120,GBP2,GBP2,GBP2|NM_004120
2,NM_138636,TLR8,TLR8,TLR8|NM_138636
3,NM_001146055,SNCA,SNCA,SNCA|NM_001146055
4,NM_001165,BIRC3,BIRC3,BIRC3|NM_001165
5,NM_005516,HLA-E,HLA-E,HLA-E|NM_005516
6,NM_001135652,EIF2AK2,EIF2AK2,EIF2AK2|NM_001135652
7,NM_000972,RPL7A,RPL7A,RPL7A|NM_000972


In [15]:
mondor3 = pd.concat([mondor1, mondor2], axis=0, ignore_index=True)
print(mondor3.shape)
display(mondor3.head(8))

(1568, 4)


,ProbeID,Probe.Label,Official.Gene.Name,"(Gene name, RefSeq mRNA ID)"
0,NM_000077,CDKN2A,CDKN2A,CDKN2A|NM_000077
1,NM_004120,GBP2,GBP2,GBP2|NM_004120
2,NM_138636,TLR8,TLR8,TLR8|NM_138636
3,NM_001146055,SNCA,SNCA,SNCA|NM_001146055
4,NM_001165,BIRC3,BIRC3,BIRC3|NM_001165
5,NM_005516,HLA-E,HLA-E,HLA-E|NM_005516
6,NM_001135652,EIF2AK2,EIF2AK2,EIF2AK2|NM_001135652
7,NM_000972,RPL7A,RPL7A,RPL7A|NM_000972


In [16]:
mondor3.drop_duplicates(inplace=True)
print(mondor3.shape)
display(mondor3.head(8))

(829, 4)


,ProbeID,Probe.Label,Official.Gene.Name,"(Gene name, RefSeq mRNA ID)"
0,NM_000077,CDKN2A,CDKN2A,CDKN2A|NM_000077
1,NM_004120,GBP2,GBP2,GBP2|NM_004120
2,NM_138636,TLR8,TLR8,TLR8|NM_138636
3,NM_001146055,SNCA,SNCA,SNCA|NM_001146055
4,NM_001165,BIRC3,BIRC3,BIRC3|NM_001165
5,NM_005516,HLA-E,HLA-E,HLA-E|NM_005516
6,NM_001135652,EIF2AK2,EIF2AK2,EIF2AK2|NM_001135652
7,NM_000972,RPL7A,RPL7A,RPL7A|NM_000972


In [17]:
# list(mondor3["(Gene name, RefSeq mRNA ID)"].str.contains("\Anan\|"))
drop_indice = mondor3.index[np.where(mondor3["(Gene name, RefSeq mRNA ID)"].str.contains("\Anan\|"))[0]]
mondor3.drop(drop_indice, inplace=True)
print(mondor3.shape)
display(mondor3.head(8))

(796, 4)


,ProbeID,Probe.Label,Official.Gene.Name,"(Gene name, RefSeq mRNA ID)"
0,NM_000077,CDKN2A,CDKN2A,CDKN2A|NM_000077
1,NM_004120,GBP2,GBP2,GBP2|NM_004120
2,NM_138636,TLR8,TLR8,TLR8|NM_138636
3,NM_001146055,SNCA,SNCA,SNCA|NM_001146055
4,NM_001165,BIRC3,BIRC3,BIRC3|NM_001165
5,NM_005516,HLA-E,HLA-E,HLA-E|NM_005516
6,NM_001135652,EIF2AK2,EIF2AK2,EIF2AK2|NM_001135652
7,NM_000972,RPL7A,RPL7A,RPL7A|NM_000972


In [18]:
merge1 = pd.merge(mondor3, ref0, how='left', on="(Gene name, RefSeq mRNA ID)")
merge1.drop(np.where(merge1["Gene name"].isna())[0], inplace=True)
merge1["NCBI gene ID"] = merge1["NCBI gene ID"].astype("Int64")
print(merge1.shape)
display(merge1.head(8))

(760, 8)


,ProbeID,Probe.Label,Official.Gene.Name,"(Gene name, RefSeq mRNA ID)",Gene name,NCBI gene ID,RefSeq mRNA ID,"(Gene name, NCBI gene ID)"
0,NM_000077,CDKN2A,CDKN2A,CDKN2A|NM_000077,CDKN2A,1029,NM_000077,CDKN2A|1029
1,NM_004120,GBP2,GBP2,GBP2|NM_004120,GBP2,2634,NM_004120,GBP2|2634
2,NM_138636,TLR8,TLR8,TLR8|NM_138636,TLR8,51311,NM_138636,TLR8|51311
3,NM_001146055,SNCA,SNCA,SNCA|NM_001146055,SNCA,6622,NM_001146055,SNCA|6622
4,NM_001165,BIRC3,BIRC3,BIRC3|NM_001165,BIRC3,330,NM_001165,BIRC3|330
5,NM_005516,HLA-E,HLA-E,HLA-E|NM_005516,HLA-E,3133,NM_005516,HLA-E|3133
6,NM_001135652,EIF2AK2,EIF2AK2,EIF2AK2|NM_001135652,EIF2AK2,5610,NM_001135652,EIF2AK2|5610
7,NM_000972,RPL7A,RPL7A,RPL7A|NM_000972,RPL7A,6130,NM_000972,RPL7A|6130


In [19]:
merge2 = pd.merge(merge1, firehose_data, how='left', left_on="(Gene name, NCBI gene ID)", right_on="Hybridization REF")
merge2.drop(np.where(merge2["Hybridization REF"].isna())[0], inplace=True)
merge2["Gene ID"] = merge2["Gene ID"].astype("Int64")
print(merge2.shape)
display(merge2.head(8))

(742, 434)


,ProbeID,Probe.Label,Official.Gene.Name,"(Gene name, RefSeq mRNA ID)",Gene name,NCBI gene ID,RefSeq mRNA ID,"(Gene name, NCBI gene ID)",Gene Name,Gene ID,...,TCGA-ZP-A9CZ-01A-11R-A38B-07 normalized_count,TCGA-ZP-A9D0-01A-11R-A37K-07 normalized_count,TCGA-ZP-A9D1-01A-11R-A38B-07 normalized_count,TCGA-ZP-A9D2-01A-11R-A38B-07 normalized_count,TCGA-ZP-A9D4-01A-11R-A37K-07 normalized_count,TCGA-ZS-A9CD-01A-11R-A37K-07 normalized_count,TCGA-ZS-A9CE-01A-11R-A37K-07 normalized_count,TCGA-ZS-A9CF-01A-11R-A38B-07 normalized_count,TCGA-ZS-A9CF-02A-11R-A38B-07 normalized_count,TCGA-ZS-A9CG-01A-11R-A37K-07 normalized_count
0,NM_000077,CDKN2A,CDKN2A,CDKN2A|NM_000077,CDKN2A,1029,NM_000077,CDKN2A|1029,CDKN2A,1029,...,831.2898,12.2265,180.0643,214.3377,417.4798,177.3103,203.5011,1241.9021,483.3127,115.8526
1,NM_004120,GBP2,GBP2,GBP2|NM_004120,GBP2,2634,NM_004120,GBP2|2634,GBP2,2634,...,3089.5546,3340.7915,2271.6977,1205.3239,297.2027,2364.2675,1662.3851,2497.7464,2851.3782,1179.5099
2,NM_138636,TLR8,TLR8,TLR8|NM_138636,TLR8,51311,NM_138636,TLR8|51311,TLR8,51311,...,55.8746,18.6615,18.0064,28.0204,11.8795,17.2802,0.5470,23.4321,13.5970,6.1697
3,NM_001146055,SNCA,SNCA,SNCA|NM_001146055,SNCA,6622,NM_001146055,SNCA|6622,SNCA,6622,...,8.6916,5.1480,3.8585,12.3726,0.5657,7.5131,0.5470,10.3377,2.4722,6.8552
4,NM_001165,BIRC3,BIRC3,BIRC3|NM_001165,BIRC3,330,NM_001165,BIRC3|330,BIRC3,330,...,1060.3756,199.4852,7423.1511,1609.1703,391.4581,2708.4899,240.1532,957.9600,580.9642,132.3051
5,NM_005516,HLA-E,HLA-E,HLA-E|NM_005516,HLA-E,3133,NM_005516,HLA-E|3133,HLA-E,3133,...,7106.6273,4638.3526,8788.4244,6938.8646,5327.1107,15572.5019,10625.2735,12548.5872,7804.0791,8815.7669
6,NM_001135652,EIF2AK2,EIF2AK2,EIF2AK2|NM_001135652,EIF2AK2,5610,NM_001135652,EIF2AK2|5610,EIF2AK2,5610,...,373.1181,225.8687,402.5723,403.9301,205.9115,187.0774,259.8468,196.4163,237.3300,175.4927
7,NM_000972,RPL7A,RPL7A,RPL7A|NM_000972,RPL7A,6130,NM_000972,RPL7A|6130,RPL7A,6130,...,5295.6697,5712.3552,8252.0900,3124.4541,13519.4456,11524.4177,15185.9956,8403.8594,9011.7429,15590.7455


In [20]:
merge2["Hybridization REF"].isna().sum()

0

In [21]:
n_not_matched_1=0
l_not_matched_1=[]
names_not_matched_1=[]
for i in mondor["ProbeID"].tolist():
    if i not in merge2["ProbeID"].tolist():
        if not i.startswith(("ERCC_")):
            n_not_matched_1=n_not_matched_1+1
            l_not_matched_1.append(i)
            row = mondor[mondor3["ProbeID"]==i]
            display(row)
            names_not_matched_1.append(row["Probe.Label"].values[0])
            names_not_matched_1.append(row["Official.Gene.Name"].values[0])
print(n_not_matched_1)
print(l_not_matched_1)

C:\Program Files\Anaconda3\envs\r-tensorflow\lib\site-packages\ipykernel\__main__.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,ProbeID,Probe.Label,Official.Gene.Name,"(Probe.Label, RefSeq mRNA ID)","(Official.Gene.Name, RefSeq mRNA ID)"
51,NM_002200,IRF5,NaN,IRF5|NM_002200,nan|NM_002200


,ProbeID,Probe.Label,Official.Gene.Name,"(Probe.Label, RefSeq mRNA ID)","(Official.Gene.Name, RefSeq mRNA ID)"
76,NM_001206796,PKM,PKM,PKM|NM_001206796,PKM|NM_001206796


,ProbeID,Probe.Label,Official.Gene.Name,"(Probe.Label, RefSeq mRNA ID)","(Official.Gene.Name, RefSeq mRNA ID)"
94,NM_002105,H2AFX,H2AFX,H2AFX|NM_002105,H2AFX|NM_002105


,ProbeID,Probe.Label,Official.Gene.Name,"(Probe.Label, RefSeq mRNA ID)","(Official.Gene.Name, RefSeq mRNA ID)"
117,NM_022153,VSIR,C10ORF54,VSIR|NM_022153,C10ORF54|NM_022153


...

,ProbeID,Probe.Label,Official.Gene.Name,"(Probe.Label, RefSeq mRNA ID)","(Official.Gene.Name, RefSeq mRNA ID)"
666,NM_007108,ELOB,TCEB2,ELOB|NM_007108,TCEB2|NM_007108


,ProbeID,Probe.Label,Official.Gene.Name,"(Probe.Label, RefSeq mRNA ID)","(Official.Gene.Name, RefSeq mRNA ID)"
673,NM_024689,CXORF36,CXORF36,CXORF36|NM_024689,CXORF36|NM_024689


,ProbeID,Probe.Label,Official.Gene.Name,"(Probe.Label, RefSeq mRNA ID)","(Official.Gene.Name, RefSeq mRNA ID)"
685,NM_000034,ALDOA,ALDOA,ALDOA|NM_000034,ALDOA|NM_000034


,ProbeID,Probe.Label,Official.Gene.Name,"(Probe.Label, RefSeq mRNA ID)","(Official.Gene.Name, RefSeq mRNA ID)"
701,NM_000584,CXCL8,CXCL8,CXCL8|NM_000584,CXCL8|NM_000584


28
['NM_002200', 'NM_001206796', 'NM_002105', 'NM_022153', 'NM_017970', 'NM_014584', 'NR_024115', 'NM_002855', 'NM_138441', 'NM_000247', 'NM_001330347', 'NR_024168', 'NR_048564', 'NM_001256252', 'ENST00000367367', 'NM_001017973', 'NM_198282', 'XM_011545241', 'NM_023914', 'NM_147162', 'XM_011542270', 'NM_001283012', 'NR_026800', 'NM_001042724', 'NM_007108', 'NM_024689', 'NM_000034', 'NM_000584']


In [22]:
names_not_matched_1 = set(filter(lambda x: x == x , set(names_not_matched_1)))

In [23]:
rematch_name = mondor3.iloc[np.where(pd.Series([i.split("|")[0] for i in mondor3["(Gene name, RefSeq mRNA ID)"]]).isin(names_not_matched_1))[0]]
rematch_name.reset_index(inplace=True)
rematch_name.drop(columns="index", inplace=True)

C:\Program Files\Anaconda3\envs\r-tensorflow\lib\site-packages\pandas\core\frame.py:4097: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [24]:
rematch_name["Gene name"] = pd.DataFrame([i.split("|")[0] for i in rematch_name["(Gene name, RefSeq mRNA ID)"]])
rematch_name

C:\Program Files\Anaconda3\envs\r-tensorflow\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,ProbeID,Probe.Label,Official.Gene.Name,"(Gene name, RefSeq mRNA ID)",Gene name
0,NM_002200,IRF5,NaN,IRF5|NM_002200,IRF5
1,NM_001206796,PKM,PKM,PKM|NM_001206796,PKM
2,NM_002105,H2AFX,H2AFX,H2AFX|NM_002105,H2AFX
3,NM_022153,VSIR,C10ORF54,VSIR|NM_022153,VSIR
4,NM_017970,NRDE2,NRDE2,NRDE2|NM_017970,NRDE2
5,NM_014584,ERO1A,ERO1A,ERO1A|NM_014584,ERO1A
6,NR_024115,GIMAP6,NaN,GIMAP6|NR_024115,GIMAP6
7,NM_002855,NECTIN1,PVRL1,NECTIN1|NM_002855,NECTIN1
8,NM_138441,MB21D1,MB21D1,MB21D1|NM_138441,MB21D1
9,NM_000247,MICA,MICA,MICA|NM_000247,MICA


In [25]:
rematch_name = pd.merge(rematch_name, firehose_data, how='left', left_on="Gene name", right_on="Gene Name")
rematch_name.drop(np.where(rematch_name["Gene ID"].isna())[0], inplace=True)
rematch_name["Gene ID"] = rematch_name["Gene ID"].astype("Int64")
print(rematch_name.shape)
display(rematch_name.head(5))
print("Using gene name, duplicate matches: "+str(rematch_name["ProbeID"].duplicated(keep=False).sum()))

(19, 431)


,ProbeID,Probe.Label,Official.Gene.Name,"(Gene name, RefSeq mRNA ID)",Gene name,Gene Name,Gene ID,Hybridization REF,TCGA-2V-A95S-01A-11R-A37K-07 normalized_count,TCGA-2Y-A9GS-01A-12R-A38B-07 normalized_count,...,TCGA-ZP-A9CZ-01A-11R-A38B-07 normalized_count,TCGA-ZP-A9D0-01A-11R-A37K-07 normalized_count,TCGA-ZP-A9D1-01A-11R-A38B-07 normalized_count,TCGA-ZP-A9D2-01A-11R-A38B-07 normalized_count,TCGA-ZP-A9D4-01A-11R-A37K-07 normalized_count,TCGA-ZS-A9CD-01A-11R-A37K-07 normalized_count,TCGA-ZS-A9CE-01A-11R-A37K-07 normalized_count,TCGA-ZS-A9CF-01A-11R-A38B-07 normalized_count,TCGA-ZS-A9CF-02A-11R-A38B-07 normalized_count,TCGA-ZS-A9CG-01A-11R-A37K-07 normalized_count
0,NM_002200,IRF5,NaN,IRF5|NM_002200,IRF5,IRF5,3663,IRF5|3663,377.9117,592.4101,...,564.3334,456.8855,338.9068,229.6215,225.7106,222.3892,332.6039,135.7684,125.4635,161.7823
2,NM_002105,H2AFX,H2AFX,H2AFX|NM_002105,H2AFX,H2AFX,3014,H2AFX|3014,1266.0042,926.0720,...,852.3980,124.1956,335.0482,439.2285,432.1878,596.5440,275.7112,1359.7519,1173.0532,394.8586
6,NR_024115,GIMAP6,NaN,GIMAP6|NR_024115,GIMAP6,GIMAP6,474344,GIMAP6|474344,464.5708,293.2479,...,353.8724,222.0077,524.7588,234.3523,230.8019,247.1826,96.8271,199.8622,260.1978,209.7686
9,NM_000247,MICA,MICA,MICA|NM_000247,MICA,MICA,4276,MICA|4276,215.0187,268.1124,...,306.6894,97.8121,162.7010,255.0946,369.9618,379.4140,342.4508,362.5086,178.6156,263.2391
11,NR_024168,TLR4,NaN,TLR4|NR_024168,TLR4,TLR4,7099,TLR4|7099,231.3080,249.3839,...,338.9725,150.5792,195.4984,201.6012,274.9258,123.9669,208.4245,117.1606,159.4561,324.9357


Using gene name, duplicate matches: 0


In [26]:
for column in list(set(merge2.columns.tolist()) - set(rematch_name.columns.tolist())):
    rematch_name[column] = np.nan
print(rematch_name.shape)
display(rematch_name.head(5))
if set(rematch_name.columns) != set(merge2.columns):
    print("Error! Something wrong with the column names.")

(19, 434)


,ProbeID,Probe.Label,Official.Gene.Name,"(Gene name, RefSeq mRNA ID)",Gene name,Gene Name,Gene ID,Hybridization REF,TCGA-2V-A95S-01A-11R-A37K-07 normalized_count,TCGA-2Y-A9GS-01A-12R-A38B-07 normalized_count,...,TCGA-ZP-A9D2-01A-11R-A38B-07 normalized_count,TCGA-ZP-A9D4-01A-11R-A37K-07 normalized_count,TCGA-ZS-A9CD-01A-11R-A37K-07 normalized_count,TCGA-ZS-A9CE-01A-11R-A37K-07 normalized_count,TCGA-ZS-A9CF-01A-11R-A38B-07 normalized_count,TCGA-ZS-A9CF-02A-11R-A38B-07 normalized_count,TCGA-ZS-A9CG-01A-11R-A37K-07 normalized_count,"(Gene name, NCBI gene ID)",RefSeq mRNA ID,NCBI gene ID
0,NM_002200,IRF5,NaN,IRF5|NM_002200,IRF5,IRF5,3663,IRF5|3663,377.9117,592.4101,...,229.6215,225.7106,222.3892,332.6039,135.7684,125.4635,161.7823,NaN,NaN,NaN
2,NM_002105,H2AFX,H2AFX,H2AFX|NM_002105,H2AFX,H2AFX,3014,H2AFX|3014,1266.0042,926.0720,...,439.2285,432.1878,596.5440,275.7112,1359.7519,1173.0532,394.8586,NaN,NaN,NaN
6,NR_024115,GIMAP6,NaN,GIMAP6|NR_024115,GIMAP6,GIMAP6,474344,GIMAP6|474344,464.5708,293.2479,...,234.3523,230.8019,247.1826,96.8271,199.8622,260.1978,209.7686,NaN,NaN,NaN
9,NM_000247,MICA,MICA,MICA|NM_000247,MICA,MICA,4276,MICA|4276,215.0187,268.1124,...,255.0946,369.9618,379.4140,342.4508,362.5086,178.6156,263.2391,NaN,NaN,NaN
11,NR_024168,TLR4,NaN,TLR4|NR_024168,TLR4,TLR4,7099,TLR4|7099,231.3080,249.3839,...,201.6012,274.9258,123.9669,208.4245,117.1606,159.4561,324.9357,NaN,NaN,NaN


In [27]:
merge3 = pd.concat([merge2,rematch_name], ignore_index=True)
print(merge3.shape)
display(merge3.head(4))

(761, 434)


C:\Program Files\Anaconda3\envs\r-tensorflow\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


,"(Gene name, NCBI gene ID)","(Gene name, RefSeq mRNA ID)",Gene ID,Gene Name,Gene name,Hybridization REF,NCBI gene ID,Official.Gene.Name,Probe.Label,ProbeID,...,TCGA-ZP-A9CZ-01A-11R-A38B-07 normalized_count,TCGA-ZP-A9D0-01A-11R-A37K-07 normalized_count,TCGA-ZP-A9D1-01A-11R-A38B-07 normalized_count,TCGA-ZP-A9D2-01A-11R-A38B-07 normalized_count,TCGA-ZP-A9D4-01A-11R-A37K-07 normalized_count,TCGA-ZS-A9CD-01A-11R-A37K-07 normalized_count,TCGA-ZS-A9CE-01A-11R-A37K-07 normalized_count,TCGA-ZS-A9CF-01A-11R-A38B-07 normalized_count,TCGA-ZS-A9CF-02A-11R-A38B-07 normalized_count,TCGA-ZS-A9CG-01A-11R-A37K-07 normalized_count
0,CDKN2A|1029,CDKN2A|NM_000077,1029,CDKN2A,CDKN2A,CDKN2A|1029,1029,CDKN2A,CDKN2A,NM_000077,...,831.2898,12.2265,180.0643,214.3377,417.4798,177.3103,203.5011,1241.9021,483.3127,115.8526
1,GBP2|2634,GBP2|NM_004120,2634,GBP2,GBP2,GBP2|2634,2634,GBP2,GBP2,NM_004120,...,3089.5546,3340.7915,2271.6977,1205.3239,297.2027,2364.2675,1662.3851,2497.7464,2851.3782,1179.5099
2,TLR8|51311,TLR8|NM_138636,51311,TLR8,TLR8,TLR8|51311,51311,TLR8,TLR8,NM_138636,...,55.8746,18.6615,18.0064,28.0204,11.8795,17.2802,0.5470,23.4321,13.5970,6.1697
3,SNCA|6622,SNCA|NM_001146055,6622,SNCA,SNCA,SNCA|6622,6622,SNCA,SNCA,NM_001146055,...,8.6916,5.1480,3.8585,12.3726,0.5657,7.5131,0.5470,10.3377,2.4722,6.8552


In [28]:
n_not_matched_2=0
l_not_matched_2=[]
id_not_matched_2=[]
for i in mondor["ProbeID"].tolist():
    if i not in merge3["ProbeID"].tolist():
        if not i.startswith(("ERCC_")):
            n_not_matched_2=n_not_matched_2+1
            l_not_matched_2.append(i)
print(n_not_matched_2)
print(l_not_matched_2)

9
['NM_001206796', 'NM_017970', 'NM_014584', 'NM_138441', 'NM_001330347', 'NM_001256252', 'ENST00000367367', 'NM_001283012', 'NM_000584']


In [29]:
rematch_id = mondor3.iloc[np.where(pd.Series([i for i in mondor3["ProbeID"]]).isin(l_not_matched_2))[0]]
rematch_id.reset_index(inplace=True)
rematch_id.drop(columns="index", inplace=True)

In [30]:
print(rematch_id.shape)
display(rematch_id)

(9, 4)


,ProbeID,Probe.Label,Official.Gene.Name,"(Gene name, RefSeq mRNA ID)"
0,NM_001206796,PKM,PKM,PKM|NM_001206796
1,NM_017970,NRDE2,NRDE2,NRDE2|NM_017970
2,NM_014584,ERO1A,ERO1A,ERO1A|NM_014584
3,NM_138441,MB21D1,MB21D1,MB21D1|NM_138441
4,NM_001330347,MRE11,NaN,MRE11|NM_001330347
5,NM_001256252,ADGRE1,ADGRE1,ADGRE1|NM_001256252
6,ENST00000367367,CD45RB,NaN,CD45RB|ENST00000367367
7,NM_001283012,DEPTOR,DEPTOR,DEPTOR|NM_001283012
8,NM_000584,CXCL8,CXCL8,CXCL8|NM_000584


In [31]:
rematch_id.drop("(Gene name, RefSeq mRNA ID)", 1, inplace=True)
rematch_id = pd.merge(rematch_id, ref0, how='left', left_on="ProbeID", right_on="RefSeq mRNA ID")
rematch_id.drop(np.where(rematch_id["NCBI gene ID"].isna())[0], inplace=True)
rematch_id["NCBI gene ID"] = rematch_id["NCBI gene ID"].astype("Int64")
print(rematch_id.shape)
display(rematch_id.head(5))
print("Using gene id, duplicate matches: "+str(rematch_id["ProbeID"].duplicated(keep=False).sum()))

(8, 8)


,ProbeID,Probe.Label,Official.Gene.Name,Gene name,NCBI gene ID,RefSeq mRNA ID,"(Gene name, NCBI gene ID)","(Gene name, RefSeq mRNA ID)"
0,NM_001206796,PKM,PKM,PKM,5315,NM_001206796,PKM|5315,PKM|NM_001206796
1,NM_017970,NRDE2,NRDE2,NRDE2,55051,NM_017970,NRDE2|55051,NRDE2|NM_017970
2,NM_014584,ERO1A,ERO1A,ERO1A,30001,NM_014584,ERO1A|30001,ERO1A|NM_014584
3,NM_138441,MB21D1,MB21D1,CGAS,115004,NM_138441,CGAS|115004,CGAS|NM_138441
4,NM_001330347,MRE11,NaN,MRE11,4361,NM_001330347,MRE11|4361,MRE11|NM_001330347


Using gene id, duplicate matches: 0


In [32]:
rematch_id = pd.merge(rematch_id, firehose_data, how='left', left_on="NCBI gene ID", right_on="Gene ID")
rematch_id.drop(np.where(rematch_id["Hybridization REF"].isna())[0], inplace=True)
print(rematch_id.shape)
display(rematch_id.head(8))

(8, 434)


,ProbeID,Probe.Label,Official.Gene.Name,Gene name,NCBI gene ID,RefSeq mRNA ID,"(Gene name, NCBI gene ID)","(Gene name, RefSeq mRNA ID)",Gene Name,Gene ID,...,TCGA-ZP-A9CZ-01A-11R-A38B-07 normalized_count,TCGA-ZP-A9D0-01A-11R-A37K-07 normalized_count,TCGA-ZP-A9D1-01A-11R-A38B-07 normalized_count,TCGA-ZP-A9D2-01A-11R-A38B-07 normalized_count,TCGA-ZP-A9D4-01A-11R-A37K-07 normalized_count,TCGA-ZS-A9CD-01A-11R-A37K-07 normalized_count,TCGA-ZS-A9CE-01A-11R-A37K-07 normalized_count,TCGA-ZS-A9CF-01A-11R-A38B-07 normalized_count,TCGA-ZS-A9CF-02A-11R-A38B-07 normalized_count,TCGA-ZS-A9CG-01A-11R-A37K-07 normalized_count
0,NM_001206796,PKM,PKM,PKM,5315,NM_001206796,PKM|5315,PKM|NM_001206796,PKM2,5315,...,6134.4094,1511.5830,2192.9260,11671.3974,438.9761,1460.5560,654.2670,1445.2102,1145.2410,1109.8543
1,NM_017970,NRDE2,NRDE2,NRDE2,55051,NM_017970,NRDE2|55051,NRDE2|NM_017970,C14ORF102,55051,...,410.9887,362.9344,169.1318,134.6434,440.6732,316.3035,622.5383,491.3853,485.7849,270.7798
2,NM_014584,ERO1A,ERO1A,ERO1A,30001,NM_014584,ERO1A|30001,ERO1A|NM_014584,ERO1L,30001,...,980.2887,1277.3488,1338.2637,816.2300,613.2089,740.0451,742.8884,1138.5252,592.0890,643.0163
3,NM_138441,MB21D1,MB21D1,CGAS,115004,NM_138441,CGAS|115004,CGAS|NM_138441,C6ORF150,115004,...,54.0121,12.8700,29.5820,47.3071,8.4854,3.7566,8.2057,12.4052,11.1248,13.0249
4,NM_001330347,MRE11,NaN,MRE11,4361,NM_001330347,MRE11|4361,MRE11|NM_001330347,MRE11A,4361,...,230.3275,229.0862,293.8907,586.9723,183.8495,152.5169,133.4792,286.0096,408.5290,159.0403
5,NM_001256252,ADGRE1,ADGRE1,ADGRE1,2015,NM_001256252,ADGRE1|2015,ADGRE1|NM_001256252,EMR1,2015,...,13.6582,12.2265,4.5016,4.7307,5.6569,0.7513,1.0941,5.5134,9.2707,2.0566
6,NM_001283012,DEPTOR,DEPTOR,DEPTOR,64798,NM_001283012,DEPTOR|64798,DEPTOR|NM_001283012,DEPDC6,64798,...,271.3022,615.1866,1436.0129,760.9170,1273.3701,969.9474,1356.6740,554.1006,397.4042,634.7901
7,NM_000584,CXCL8,CXCL8,CXCL8,3576,NM_000584,CXCL8|3576,CXCL8|NM_000584,IL8,3576,...,60.8412,59.8456,4560.7717,551.3100,22.0619,1345.6048,2.1882,106.1337,41.4091,110.3685


In [33]:
for column in list(set(merge2.columns.tolist()) - set(rematch_id.columns.tolist())):
    rematch_id[column] = np.nan
print(rematch_id.shape)
display(rematch_id.head(5))
if set(rematch_id.columns) != set(merge2.columns):
    print("Error! Something wrong with the column names.")

(8, 434)


,ProbeID,Probe.Label,Official.Gene.Name,Gene name,NCBI gene ID,RefSeq mRNA ID,"(Gene name, NCBI gene ID)","(Gene name, RefSeq mRNA ID)",Gene Name,Gene ID,...,TCGA-ZP-A9CZ-01A-11R-A38B-07 normalized_count,TCGA-ZP-A9D0-01A-11R-A37K-07 normalized_count,TCGA-ZP-A9D1-01A-11R-A38B-07 normalized_count,TCGA-ZP-A9D2-01A-11R-A38B-07 normalized_count,TCGA-ZP-A9D4-01A-11R-A37K-07 normalized_count,TCGA-ZS-A9CD-01A-11R-A37K-07 normalized_count,TCGA-ZS-A9CE-01A-11R-A37K-07 normalized_count,TCGA-ZS-A9CF-01A-11R-A38B-07 normalized_count,TCGA-ZS-A9CF-02A-11R-A38B-07 normalized_count,TCGA-ZS-A9CG-01A-11R-A37K-07 normalized_count
0,NM_001206796,PKM,PKM,PKM,5315,NM_001206796,PKM|5315,PKM|NM_001206796,PKM2,5315,...,6134.4094,1511.5830,2192.9260,11671.3974,438.9761,1460.5560,654.2670,1445.2102,1145.2410,1109.8543
1,NM_017970,NRDE2,NRDE2,NRDE2,55051,NM_017970,NRDE2|55051,NRDE2|NM_017970,C14ORF102,55051,...,410.9887,362.9344,169.1318,134.6434,440.6732,316.3035,622.5383,491.3853,485.7849,270.7798
2,NM_014584,ERO1A,ERO1A,ERO1A,30001,NM_014584,ERO1A|30001,ERO1A|NM_014584,ERO1L,30001,...,980.2887,1277.3488,1338.2637,816.2300,613.2089,740.0451,742.8884,1138.5252,592.0890,643.0163
3,NM_138441,MB21D1,MB21D1,CGAS,115004,NM_138441,CGAS|115004,CGAS|NM_138441,C6ORF150,115004,...,54.0121,12.8700,29.5820,47.3071,8.4854,3.7566,8.2057,12.4052,11.1248,13.0249
4,NM_001330347,MRE11,NaN,MRE11,4361,NM_001330347,MRE11|4361,MRE11|NM_001330347,MRE11A,4361,...,230.3275,229.0862,293.8907,586.9723,183.8495,152.5169,133.4792,286.0096,408.5290,159.0403


In [34]:
merge4 = pd.concat([merge3,rematch_id], ignore_index=True)
print(merge4.shape)
display(merge4.head(4))

(769, 434)


C:\Program Files\Anaconda3\envs\r-tensorflow\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


,"(Gene name, NCBI gene ID)","(Gene name, RefSeq mRNA ID)",Gene ID,Gene Name,Gene name,Hybridization REF,NCBI gene ID,Official.Gene.Name,Probe.Label,ProbeID,...,TCGA-ZP-A9CZ-01A-11R-A38B-07 normalized_count,TCGA-ZP-A9D0-01A-11R-A37K-07 normalized_count,TCGA-ZP-A9D1-01A-11R-A38B-07 normalized_count,TCGA-ZP-A9D2-01A-11R-A38B-07 normalized_count,TCGA-ZP-A9D4-01A-11R-A37K-07 normalized_count,TCGA-ZS-A9CD-01A-11R-A37K-07 normalized_count,TCGA-ZS-A9CE-01A-11R-A37K-07 normalized_count,TCGA-ZS-A9CF-01A-11R-A38B-07 normalized_count,TCGA-ZS-A9CF-02A-11R-A38B-07 normalized_count,TCGA-ZS-A9CG-01A-11R-A37K-07 normalized_count
0,CDKN2A|1029,CDKN2A|NM_000077,1029,CDKN2A,CDKN2A,CDKN2A|1029,1029,CDKN2A,CDKN2A,NM_000077,...,831.2898,12.2265,180.0643,214.3377,417.4798,177.3103,203.5011,1241.9021,483.3127,115.8526
1,GBP2|2634,GBP2|NM_004120,2634,GBP2,GBP2,GBP2|2634,2634,GBP2,GBP2,NM_004120,...,3089.5546,3340.7915,2271.6977,1205.3239,297.2027,2364.2675,1662.3851,2497.7464,2851.3782,1179.5099
2,TLR8|51311,TLR8|NM_138636,51311,TLR8,TLR8,TLR8|51311,51311,TLR8,TLR8,NM_138636,...,55.8746,18.6615,18.0064,28.0204,11.8795,17.2802,0.5470,23.4321,13.5970,6.1697
3,SNCA|6622,SNCA|NM_001146055,6622,SNCA,SNCA,SNCA|6622,6622,SNCA,SNCA,NM_001146055,...,8.6916,5.1480,3.8585,12.3726,0.5657,7.5131,0.5470,10.3377,2.4722,6.8552


In [35]:
merge4.drop(["(Gene name, NCBI gene ID)", "Gene name", "NCBI gene ID", "RefSeq mRNA ID"], 1, inplace=True)
merge4.isna().sum()[0:10]

(Gene name, RefSeq mRNA ID)                       0
Gene ID                                           0
Gene Name                                         0
Hybridization REF                                 0
Official.Gene.Name                               18
Probe.Label                                       0
ProbeID                                           0
TCGA-2V-A95S-01A-11R-A37K-07 normalized_count     0
TCGA-2Y-A9GS-01A-12R-A38B-07 normalized_count     0
TCGA-2Y-A9GT-01A-11R-A38B-07 normalized_count     0
dtype: int64

In [36]:
list(merge4.columns)[0:10]

['(Gene name, RefSeq mRNA ID)',
 'Gene ID',
 'Gene Name',
 'Hybridization REF',
 'Official.Gene.Name',
 'Probe.Label',
 'ProbeID',
 'TCGA-2V-A95S-01A-11R-A37K-07 normalized_count',
 'TCGA-2Y-A9GS-01A-12R-A38B-07 normalized_count',
 'TCGA-2Y-A9GT-01A-11R-A38B-07 normalized_count']

In [59]:
du = np.where(merge4["Gene ID"].duplicated(keep=False))
display(merge4.iloc[du])
merge4.drop(list(du[0])[1:], inplace=True)

,"(Gene name, RefSeq mRNA ID)",Gene ID,Gene Name,Hybridization REF,Official.Gene.Name,Probe.Label,ProbeID,TCGA-2V-A95S-01A-11R-A37K-07 normalized_count,TCGA-2Y-A9GS-01A-12R-A38B-07 normalized_count,TCGA-2Y-A9GT-01A-11R-A38B-07 normalized_count,...,TCGA-ZP-A9CZ-01A-11R-A38B-07 normalized_count,TCGA-ZP-A9D0-01A-11R-A37K-07 normalized_count,TCGA-ZP-A9D1-01A-11R-A38B-07 normalized_count,TCGA-ZP-A9D2-01A-11R-A38B-07 normalized_count,TCGA-ZP-A9D4-01A-11R-A37K-07 normalized_count,TCGA-ZS-A9CD-01A-11R-A37K-07 normalized_count,TCGA-ZS-A9CE-01A-11R-A37K-07 normalized_count,TCGA-ZS-A9CF-01A-11R-A38B-07 normalized_count,TCGA-ZS-A9CF-02A-11R-A38B-07 normalized_count,TCGA-ZS-A9CG-01A-11R-A37K-07 normalized_count
682,PTPRC|NM_080921,5788,PTPRC,PTPRC|5788,PTPRC,PTPRC,NM_080921,1484.2808,367.6688,1318.0901,...,711.4698,174.3887,591.6399,383.1878,148.211,226.8971,237.4179,444.521,378.8628,163.8389
735,PTPRC|NM_002838,5788,PTPRC,PTPRC|5788,PTPRC,CD45RA,NM_002838,1484.2808,367.6688,1318.0901,...,711.4698,174.3887,591.6399,383.1878,148.211,226.8971,237.4179,444.521,378.8628,163.8389
740,PTPRC|NM_080921,5788,PTPRC,PTPRC|5788,PTPRC,CD45RO,NM_080921,1484.2808,367.6688,1318.0901,...,711.4698,174.3887,591.6399,383.1878,148.211,226.8971,237.4179,444.521,378.8628,163.8389


In [62]:
merge4.to_csv("E:\\deeplearning\\Hepatocarcinomes\\TCGA\\processed\\firehose_filtered.csv", sep='\t', index=False, header= True)